Setup dependencies and logging

In [1]:
import logging
import json
from IPython.display import Pretty, HTML
import urllib3

from pyfhirclient.FhirClient import FhirClient

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

# Disable the unverified HTTPS request warnings
urllib3.disable_warnings()

In [2]:
def setup_fhir_client() -> FhirClient:
    fhir_client = FhirClient(
        base_url='https://concept01.ehealthexchange.org:52780/fhirproxy/r4/',
        verify_ssl=False,
        extra_headers={
            'X-DESTINATION': 'PublicHAPI',
            'X-POU': 'TREAT',
            'Host': 'concept01.ehealthexchange.org:52780'
        })

    fhir_client.auth_type = 'Basic'
    fhir_client.token = 'c3ZjX2VIeEZISVJTYW5kYm94OndpbGxmdWxTdHJvbmdTdGFuZHVyZDc'

    return fhir_client

In [4]:
client = setup_fhir_client()

## TODO: Ensemble Timeout?
search_results = client.search('Patient', {
    'family': 'Smith',
    'gender': 'male',
    'birthdate': '1965-09-01',
    'address-state': 'NC',
})
patient = search_results['entry'][0]
Pretty(json.dumps(patient, indent=4))

ERROR:root:b'{\r\n  "resourceType":"OperationOutcome",\r\n  "issue":[\r\n    {\r\n      "severity":"error",\r\n      "code":"exception",\r\n      "diagnostics":"<HSFHIRErr>EnsembleTimeout",\r\n      "details":{\r\n        "text":"Ensemble reported a timeout"\r\n      }\r\n    }\r\n  ]\r\n}'


HTTPError: 500 Server Error: Internal Server Error for url: https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Patient?family=Smith&gender=male&birthdate=1965-09-01&address-state=NC

In [ ]:
HTML(patient['resource']['text']['div'])

In [ ]:
everything = client.everything('Patient', patient['resource']['id'])
Pretty(json.dumps(everything, indent=4))

In [ ]:
medication_administration_results = client.search('MedicationAdministration', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(medication_administration_results, indent=4))

In [ ]:
observation_search_results = client.search('Observation', query_params={
    'patient': patient['resource']['id']
    'code': '76272004'
})

Pretty(json.dumps(observation_search_results, indent=4))        

In [ ]:
encounter_search_results = client.search('Encounter', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(encounter_search_results, indent=4))

In [ ]:
client.close()